In [1]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
import FinanceDataReader as fdr

import datetime as dt

import numpy as np

from tqdm import tqdm #진행바
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

# GPU
import tensorflow as tf

#https://www.tensorflow.org/guide/gpu#allowing_gpu_memory_growth
#프로세스의 요구량만큼 메모리 사용 설정
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        for i in range(len(gpus)):
            tf.config.experimental.set_memory_growth(gpus[i], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)
        
#지정해서 쓰고 싶으면 model 쓸 때는 with 구문 안에 넣기

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [2]:
df = pd.read_csv('stock_2018_10days.csv')
# x = pd.read_csv('capstone_x.csv')
# # y = 

In [12]:
# x = df.iloc[:,:-1]
# y = df.iloc[:,-1]

In [3]:
# Next_Change 열 값 조건에 따라 변경
# 열 값은 왜 안바뀌는 가에 대하여...
# df[df.iloc[:,-1]>0.05].iloc[:,-1] = 1
# df[df.iloc[:,-1]<=0.05].iloc[:,-1] = 0
# df.iloc[:,-1]

# df[(df['Next_Change']>0.05)]['Next_Change'] = 1 
# df[(df['Next_Change']<=0.05)]['Next_Change'] = 0
# df

# 1 if df[df.iloc[:,-1]>0.05].iloc[:,-1] else 0

df.loc[(df.Next_Change>0.05),'Next_Change'] = 1
df.loc[(df.Next_Change<=0.05),'Next_Change'] = 0
df
# https://pongdangstory.tistory.com/518 한줄에 왜 안끝나?

,Code,Date,D-9_Open,D-9_High,D-9_Low,D-9_Close,D-9_Volume,D-9_Trading_Value,D-9_Change,D-9_MA5,...,D-0_TSI,D-0_UO,D-0_SR,D-0_WR,D-0_AO,D-0_KAMA,D-0_ROC,D-0_PPO,D-0_PVO,Next_Change
0,50120,2018-01-15,10250,12050,10150,11800,26086769,307823874200,0.145631,10108.0,...,29.3225,45.3787,63.9228,-36.07720,3645.880000,11858.8,68.67470,12.32460,38.8520,0.0
1,50120,2018-01-16,11950,12450,10900,11750,20460474,240410569500,-0.004237,10538.0,...,28.0689,45.6406,53.2520,-46.74800,3980.740000,11947.6,47.78580,12.29020,30.6275,0.0
2,50120,2018-01-17,11850,14150,11600,12600,60663854,764364560400,0.072340,11232.0,...,22.3139,41.0354,25.8130,-74.18700,3687.650000,11946.2,13.10680,10.44330,23.7000,0.0
3,50120,2018-01-18,12800,13200,12000,12200,13935258,170010147600,-0.031746,11730.0,...,18.8794,41.3833,28.8725,-71.12750,2987.650000,11947.9,3.38983,9.22399,17.4172,0.0
4,50120,2018-01-19,12450,13400,12350,12850,16471707,211661434950,0.053279,12240.0,...,16.5383,33.5502,25.0000,-75.00000,2637.350000,11949.9,5.10638,8.26069,12.6972,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2532,11760,2018-05-15,21400,24100,20800,22400,1488904,33351449600,0.095354,20830.0,...,56.6722,68.0251,84.6369,-15.36310,6866.910000,27722.5,74.93730,13.01120,68.4751,0.0
2533,11760,2018-05-17,23500,23850,21950,22350,489326,10936436100,-0.064854,22040.0,...,54.5403,64.0099,85.1942,-14.80580,10338.970588,29517.6,83.12960,15.48880,67.1178,0.0
2534,11760,2018-06-07,41650,41900,37000,38800,1305256,50643932800,-0.058252,39190.0,...,29.5457,49.8274,70.6329,-29.36710,10088.100000,36598.8,11.81820,13.02900,17.8270,0.0
2535,11760,2018-06-11,40500,41300,40050,41300,712521,29427117300,0.298742,38620.0,...,31.8637,53.8917,95.8333,-4.16667,10979.700000,39093.7,20.87380,13.90680,19.4140,0.0


## 머신러닝 K-fold 교차 검증(cross validation)
데이터를 5-fold 교차검증으로 8개 모델을 학습하고 예측하여 평가한다.

### 1. Logistic Regression

In [4]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import time

results=[]
X = df.iloc[:,2:-1]
# X
y = df.iloc[:,-1]
# y
i=0
skf = StratifiedKFold(n_splits=5)
for train_idx, test_idx in skf.split(X,y):
#     train_idx, test_idx
    x_train = df.iloc[train_idx,2:-1]
    y_train = df.iloc[train_idx,-1]
    x_test = df.iloc[test_idx,2:-1]
    y_test = df.iloc[test_idx,-1]
    # 1. Logistic regression    
    from sklearn.linear_model import LogisticRegression
    model = LogisticRegression(max_iter=1000)
    start_time = time.time()
    _ = model.fit(x_train, y_train) # 학습
    
    training_time = time.time() - start_time
    
    y_pred_train = model.predict(x_train) 
    y_pred_test = model.predict(x_test) # 예측
    accuracy_train = accuracy_score(y_train, y_pred_train) # 설명변수 x_train에 대한 반응변수 y_train 예측의 정확도. 이미 학습할 때 사용한 데이터이므로 답을 아는 상태라 소용이 없다고 생각하면 됨.
    accuracy_test = accuracy_score(y_test, y_pred_test) # 학습에 사용하지 않은 설명변수 x_train에 대한 반응변수 y_test 예측의 정확도. 이게 늘 말하는 그 정확도임.
    results.append([i,'LogisticRegression',accuracy_train,accuracy_test, training_time])
    i+=1
results
# # 형태: i번째 학습, 학습 모델, train 정확도, test 정확도(최종정확도) 학습에 걸린 시간

[[0,
  'LogisticRegression',
  0.917200591424347,
  0.9173228346456693,
  0.016400814056396484],
 [1,
  'LogisticRegression',
  0.917200591424347,
  0.9173228346456693,
  0.018486738204956055],
 [2,
  'LogisticRegression',
  0.9172413793103448,
  0.9171597633136095,
  0.01621866226196289],
 [3,
  'LogisticRegression',
  0.9172413793103448,
  0.9171597633136095,
  0.01723170280456543],
 [4,
  'LogisticRegression',
  0.9172413793103448,
  0.9171597633136095,
  0.018756866455078125]]

In [38]:
results = np.array(results)
results
results.shape

results[:,3] #test 정확도
(results[:,3]).astype(float)
np.mean((results[:,3]).astype(float))

# cannot perform reduce with flexible type ; 숫자 아닌 애들로 수치연산 하려고 하면 발생하는 에러.
# https://raisonde.tistory.com/entry/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%97%90%EB%9F%AC-TypeError-cannot-perform-reduce-with-flexible-type

# only size-1 arrays can be converted to Python scalars ; ndarray 안의 여러 값들을 int로 바꾸려할 때 발생하는 에러.
# 해결책: x.astype(int)
# https://www.statology.org/only-size-1-arrays-can-be-converted-to-python-scalars/
# np.mean(list(results[:,3]))

array([['0', 'LogisticRegression', '0.917200591424347',
        '0.9173228346456693', '0.016400814056396484'],
       ['1', 'LogisticRegression', '0.917200591424347',
        '0.9173228346456693', '0.018486738204956055'],
       ['2', 'LogisticRegression', '0.9172413793103448',
        '0.9171597633136095', '0.01621866226196289'],
       ['3', 'LogisticRegression', '0.9172413793103448',
        '0.9171597633136095', '0.01723170280456543'],
       ['4', 'LogisticRegression', '0.9172413793103448',
        '0.9171597633136095', '0.018756866455078125']], dtype='<U21')

(5, 5)

array(['0.9173228346456693', '0.9173228346456693', '0.9171597633136095',
       '0.9171597633136095', '0.9171597633136095'], dtype='<U21')

array([0.91732283, 0.91732283, 0.91715976, 0.91715976, 0.91715976])

0.9172249918464335

### 2. Decision tree

In [73]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

X = df.iloc[:,2:-1]
y = df.iloc[:,-1]
i=0
skf = StratifiedKFold(n_splits=5)
for train_idx, test_idx in skf.split(X,y):
    x_train = df.iloc[train_idx,2:-1]
    y_train = df.iloc[train_idx,-1]
    x_test = df.iloc[test_idx,2:-1]
    y_test = df.iloc[test_idx,-1]
    # 2. Decision tree
    from sklearn.tree import DecisionTreeClassifier
    model = DecisionTreeClassifier()
    start_time = time.time()    
    _ = model.fit(x_train, y_train)
    training_time = time.time() - start_time    
    y_pred_train = model.predict(x_train)
    y_pred_test = model.predict(x_test)
    accuracy_train = accuracy_score(y_train, y_pred_train)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    results.append([i,'DecisionTree',accuracy_train,accuracy_test, training_time])
    i+=1
# results[-5:]
results = np.array(results)
np.mean((results[:,3]).astype(float))

[[0, 'DecisionTree', 1.0, 0.7106299212598425, 1.7023417949676514],
 [1, 'DecisionTree', 1.0, 0.781496062992126, 1.0880887508392334],
 [2, 'DecisionTree', 1.0, 0.5522682445759369, 1.629640817642212],
 [3, 'DecisionTree', 1.0, 0.8264299802761341, 1.128674030303955],
 [4, 'DecisionTree', 1.0, 0.8777120315581854, 1.6759753227233887]]

### 3. Support vector machine

In [74]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

X = df.iloc[:,2:-1]
y = df.iloc[:,-1]
i=0
skf = StratifiedKFold(n_splits=5)
for train_idx, test_idx in skf.split(X,y):
    x_train = df.iloc[train_idx,2:-1]
    y_train = df.iloc[train_idx,-1]
    x_test = df.iloc[test_idx,2:-1]
    y_test = df.iloc[test_idx,-1]
    # 3. Support vector machine
    from sklearn.svm import SVC
    model = SVC()
    start_time = time.time()
    _ = model.fit(x_train, y_train)
    training_time = time.time() - start_time
    y_pred_train = model.predict(x_train)
    y_pred_test = model.predict(x_test)
    accuracy_train = accuracy_score(y_train, y_pred_train)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    results.append([i,'SupportVectorMachine',accuracy_train,accuracy_test, training_time])
    i+=1
# results[-5:]
results = np.array(results)
np.mean((results[:,3]).astype(float))

SVC()

SVC()

SVC()

SVC()

SVC()

[[0,
  'SupportVectorMachine',
  0.917200591424347,
  0.9173228346456693,
  0.12297511100769043],
 [1,
  'SupportVectorMachine',
  0.917200591424347,
  0.9173228346456693,
  0.12559175491333008],
 [2,
  'SupportVectorMachine',
  0.9172413793103448,
  0.9171597633136095,
  0.11754012107849121],
 [3,
  'SupportVectorMachine',
  0.9177339901477832,
  0.9171597633136095,
  0.12907767295837402],
 [4,
  'SupportVectorMachine',
  0.9172413793103448,
  0.9171597633136095,
  0.12760472297668457]]